In [143]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def read_metrics(file_path):
    metrics = pd.read_csv(file_path)
    metrics['total_time_us'] = metrics['total_time_us'] - metrics['inference_start']
    metrics['non_max_suppression'] = metrics['non_max_suppression'] - metrics['model']
    metrics['model'] = metrics['model'] - metrics['inference_start']

    metrics['inference_start'] = metrics['inference_start'] - metrics['inference_start'][0]

    metrics.iloc(1)[:4] = metrics.iloc(1)[:4] // 1000
    metrics = metrics[metrics['model'] < metrics['model'].quantile(0.99)]
    metrics['model_per_frame'] = metrics['model'] // metrics['batch_size']
    metrics['total_per_frame'] = metrics['total_time_us'] // metrics['batch_size']
    return metrics

def create_graph(metrics_list, name_list, title):
    fig = make_subplots(rows=len(metrics_list), cols=1, shared_xaxes=True, vertical_spacing=0.02)
    for idx, (metric, name) in enumerate(zip(metrics_list, name_list)):
        fig.add_histogram(x=metric['total_per_frame'], row=idx+1, col=1, nbinsx=200, name=name)
    fig.update_layout(height=100*len(metrics_list)+150, title=title)
    return fig
    

In [144]:
files = [
    'remote/metrics_yolov8n_b1_cpu_6t.csv',
    'remote/metrics_yolov8n_b1_cpu_12t.csv',
    'remote/metrics_yolov8n_b1_cpu_24t.csv',
    'remote/metrics_yolov8n_b1_cpu_36t.csv',
    'remote/metrics_yolov8n_b1_cpu_48t.csv',
    'remote/metrics_yolov8n_b1_cpu_112t.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Thread scaling - Batch size 1').show()

files = [
    'remote/metrics_yolov8n_b3_cpu_6t.csv',
    'remote/metrics_yolov8n_b3_cpu_12t.csv',
    'remote/metrics_yolov8n_b3_cpu_24t.csv',
    'remote/metrics_yolov8n_b3_cpu_36t.csv',
    'remote/metrics_yolov8n_b3_cpu_48t.csv',
    'remote/metrics_yolov8n_b3_cpu_112t.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Thread scaling - Batch size 3').show()

files = [
    'remote/metrics_yolov8n_b5_cpu_6t.csv',
    'remote/metrics_yolov8n_b5_cpu_12t.csv',
    'remote/metrics_yolov8n_b5_cpu_24t.csv',
    'remote/metrics_yolov8n_b5_cpu_36t.csv',
    'remote/metrics_yolov8n_b5_cpu_48t.csv',
    'remote/metrics_yolov8n_b5_cpu_112t.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Thread scaling - Batch size 5').show()

files = [
    'remote/metrics_yolov8n_b10_cpu_6t.csv',
    'remote/metrics_yolov8n_b10_cpu_12t.csv',
    'remote/metrics_yolov8n_b10_cpu_24t.csv',
    'remote/metrics_yolov8n_b10_cpu_36t.csv',
    'remote/metrics_yolov8n_b10_cpu_48t.csv',
    'remote/metrics_yolov8n_b10_cpu_112t.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Thread scaling - Batch size 10').show()

files = [
    'remote/metrics_yolov8n_b20_cpu_6t.csv',
    'remote/metrics_yolov8n_b20_cpu_12t.csv',
    'remote/metrics_yolov8n_b20_cpu_24t.csv',
    'remote/metrics_yolov8n_b20_cpu_36t.csv',
    'remote/metrics_yolov8n_b20_cpu_48t.csv',
    'remote/metrics_yolov8n_b20_cpu_112t.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Thread scaling - Batch size 20').show()

files = [
    'remote/metrics_yolov8n_b40_cpu_6t.csv',
    'remote/metrics_yolov8n_b40_cpu_12t.csv',
    'remote/metrics_yolov8n_b40_cpu_24t.csv',
    'remote/metrics_yolov8n_b40_cpu_36t.csv',
    'remote/metrics_yolov8n_b40_cpu_48t.csv',
    'remote/metrics_yolov8n_b40_cpu_112t.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Thread scaling - Batch size 40').show()

In [141]:
files = [
    'remote/metrics_yolov8n_b1_cpu_6t.csv',
    'remote/metrics_yolov8n_b3_cpu_6t.csv',
    'remote/metrics_yolov8n_b5_cpu_6t.csv',
    'remote/metrics_yolov8n_b10_cpu_6t.csv',
    'remote/metrics_yolov8n_b20_cpu_6t.csv',
    'remote/metrics_yolov8n_b40_cpu_6t.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Batch scaling - 6 Threads').show()

files = [
    'remote/metrics_yolov8n_b1_cpu_12t.csv',
    'remote/metrics_yolov8n_b3_cpu_12t.csv',
    'remote/metrics_yolov8n_b5_cpu_12t.csv',
    'remote/metrics_yolov8n_b10_cpu_12t.csv',
    'remote/metrics_yolov8n_b20_cpu_12t.csv',
    'remote/metrics_yolov8n_b40_cpu_12t.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Batch scaling - 12 Threads').show()

files = [
    'remote/metrics_yolov8n_b1_cpu_24t.csv',
    'remote/metrics_yolov8n_b3_cpu_24t.csv',
    'remote/metrics_yolov8n_b5_cpu_24t.csv',
    'remote/metrics_yolov8n_b10_cpu_24t.csv',
    'remote/metrics_yolov8n_b20_cpu_24t.csv',
    'remote/metrics_yolov8n_b40_cpu_24t.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Batch scaling - 24 Threads').show()

files = [
    'remote/metrics_yolov8n_b1_cpu_36t.csv',
    'remote/metrics_yolov8n_b3_cpu_36t.csv',
    'remote/metrics_yolov8n_b5_cpu_36t.csv',
    'remote/metrics_yolov8n_b10_cpu_36t.csv',
    'remote/metrics_yolov8n_b20_cpu_36t.csv',
    'remote/metrics_yolov8n_b40_cpu_36t.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Batch scaling - 36 Threads').show()

files = [
    'remote/metrics_yolov8n_b1_cpu_48t.csv',
    'remote/metrics_yolov8n_b3_cpu_48t.csv',
    'remote/metrics_yolov8n_b5_cpu_48t.csv',
    'remote/metrics_yolov8n_b10_cpu_48t.csv',
    'remote/metrics_yolov8n_b20_cpu_48t.csv',
    'remote/metrics_yolov8n_b40_cpu_48t.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Batch scaling - 48 Threads').show()

files = [
    'remote/metrics_yolov8n_b1_cpu_112t.csv',
    'remote/metrics_yolov8n_b3_cpu_112t.csv',
    'remote/metrics_yolov8n_b5_cpu_112t.csv',
    'remote/metrics_yolov8n_b10_cpu_112t.csv',
    'remote/metrics_yolov8n_b20_cpu_112t.csv',
    'remote/metrics_yolov8n_b40_cpu_112t.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Batch scaling - 112 Threads').show()

In [147]:
files = [
    'remote_parallel/metrics_yolov8n_b10_cpu_48t_1p_1.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Parallel scaling - 1 process').show()

files = [
    'remote_parallel/metrics_yolov8n_b10_cpu_48t_2p_1.csv',
    'remote_parallel/metrics_yolov8n_b10_cpu_48t_2p_2.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Parallel scaling - 2 processes').show()

files = [
    'remote_parallel/metrics_yolov8n_b10_cpu_48t_3p_1.csv',
    'remote_parallel/metrics_yolov8n_b10_cpu_48t_3p_2.csv',
    'remote_parallel/metrics_yolov8n_b10_cpu_48t_3p_3.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Parallel scaling - 3 processes').show()

files = [
    'remote_parallel/metrics_yolov8n_b10_cpu_48t_4p_1.csv',
    'remote_parallel/metrics_yolov8n_b10_cpu_48t_4p_2.csv',
    'remote_parallel/metrics_yolov8n_b10_cpu_48t_4p_3.csv',
    'remote_parallel/metrics_yolov8n_b10_cpu_48t_4p_4.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Parallel scaling - 4 processes').show()

In [150]:
files = [
    'remote_parallel/metrics_yolov8n_b5_cpu_48t_2p_1c.csv',
    'remote_parallel/metrics_yolov8n_b5_cpu_48t_2p_2c.csv',
    'remote_parallel/metrics_yolov8n_b10_cpu_48t_2p_1c.csv',
    'remote_parallel/metrics_yolov8n_b10_cpu_48t_2p_2c.csv',
    'remote_parallel/metrics_yolov8n_b20_cpu_48t_2p_1c.csv',
    'remote_parallel/metrics_yolov8n_b20_cpu_48t_2p_2c.csv',
]

create_graph([read_metrics(f) for f in files], files, 'Comparing one process on CPU 0 (1c) and one on CPU 1 (2c)').show()